# TSA Chapter 10: Unemployment Rate Forecast

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch10/TSA_ch10_unemployment_forecast/TSA_ch10_unemployment_forecast.ipynb)

This notebook demonstrates:
- Forecasting U.S. unemployment rate using ARIMA with train/test evaluation.

In [ ]:
!pip install pandas-datareader statsmodels matplotlib numpy pandas -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.arima.model import ARIMA
import pandas_datareader.data as web
import os, warnings
warnings.filterwarnings('ignore')

In [ ]:
# Color scheme and style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12,
    'axes.labelsize': 10, 'xtick.labelsize': 9, 'ytick.labelsize': 9,
    'legend.fontsize': 9, 'figure.dpi': 150, 'lines.linewidth': 1.2,
    'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')

def legend_outside(ax, ncol=3, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)

In [ ]:
# Download unemployment data and forecast with ARIMA
try:
    unemp = web.DataReader('UNRATE', 'fred', '2001-01-01', '2025-01-01')
except Exception:
    np.random.seed(42)
    dates = pd.date_range('2001-01-01', '2025-01-01', freq='MS')
    vals = 5 + 2*np.sin(np.linspace(0, 6*np.pi, len(dates))) + np.random.normal(0, 0.3, len(dates))
    unemp = pd.DataFrame({'UNRATE': np.maximum(vals, 2)}, index=dates)

y = unemp['UNRATE']
n_test = 24
train = y.iloc[:-n_test]
test = y.iloc[-n_test:]

model = ARIMA(train, order=(2, 1, 2))
res = model.fit()
fcast = res.get_forecast(steps=n_test)
pred_mean = fcast.predicted_mean
ci = fcast.conf_int(alpha=0.05)

fig, ax = plt.subplots(figsize=(8, 3.5))
ax.plot(train.index, train.values, color=COLORS['blue'], lw=0.8, label='Training data')
ax.plot(test.index, test.values, color=COLORS['gray'], lw=0.8, label='Actual (test)')
ax.plot(pred_mean.index, pred_mean.values, color=COLORS['green'], lw=1.0, label='Forecast')
ax.fill_between(ci.index, ci.iloc[:, 0], ci.iloc[:, 1], color=COLORS['green'], alpha=0.15, label='95% CI')
ax.set_xlabel('Date'); ax.set_ylabel('Unemployment Rate (%)')
ax.set_title('Forecast for U.S. Unemployment Rate')
ax.xaxis.set_major_locator(mdates.YearLocator(3))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
legend_outside(ax, ncol=2, y=-0.20)
fig.tight_layout()
save_chart(fig, 'unemployment_forecast')
plt.show()